# Estimating CAPM

- This file can estimate CAPM for any firms that Yahoo has ticker data for.
- As written, it will estimate beta separately _for each calendar year_ in the date range you give it. But you can adjust it

- _Caveat: Yahoo Finance is more of a "quick and dirty" way to get return data. WRDS has more accurate return data._



In [1]:
#!pip install pandas_datareader # uncomment and run this ONE TIME ONLY to install pandas data reader

In [2]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr # you might need to install this (see above)
from datetime import datetime

## Load asset return data 

Load your stock returns. This file uses yahoo finance.
The returns don't even have to be firms! They can be any asset. (Portfolios, mutual funds, crypto, …)

In [3]:
# choose your firms and years 
stocks = ['SBUX','AAPL','MSFT']
start  = datetime(2016, 1, 1)
end    = datetime(2016, 12, 31)

In [4]:
# download stock prices 
# here, from yahoo: not my fav source, but quick. 
# we need to do some data manipulation to get the data ready 
stock_prices = pdr.get_data_yahoo(stocks, start=start, end=end)
stock_prices = stock_prices.filter(like='Adj Close') # reduce to just columns with this in the name
stock_prices.columns = stocks # put their tickers as column names
stock_prices # uncomment to print and see

,SBUX,AAPL,MSFT
Date,,,
2016-01-04,52.108955,24.251431,49.591057
2016-01-05,52.457798,23.643709,49.817295
2016-01-06,51.992687,23.181011,48.912350
2016-01-07,50.704731,22.202665,47.211040
2016-01-08,50.651058,22.320063,47.355850
...,...,...,...
2016-12-23,51.753979,27.419535,58.799175
2016-12-27,51.617821,27.593678,58.836361
2016-12-28,51.154823,27.476017,58.566723


In [5]:
# this is wide data... so if we want to create a new variable, 
# then we have to do it once for each firm...
# what if we have 1000 firms? seems tough to do...
# make long/tidy:
stock_prices = stock_prices.stack().swaplevel().sort_index().reset_index()
stock_prices.columns = ['Firm','Date','Adj Close']
stock_prices # print - now that is formatted nicely, like CRSP! 

,Firm,Date,Adj Close
0,AAPL,2016-01-04,24.251431
1,AAPL,2016-01-05,23.643709
2,AAPL,2016-01-06,23.181011
3,AAPL,2016-01-07,22.202665
4,AAPL,2016-01-08,22.320063
...,...,...,...
751,SBUX,2016-12-23,51.753979
752,SBUX,2016-12-27,51.617821
753,SBUX,2016-12-28,51.154823
754,SBUX,2016-12-29,51.127586


In [6]:
# add return var = pct_change() function compares to prior row
# EXCEPT: don't compare for first row of one firm with last row of prior firm!
# MAKE SURE YOU CREATE THE VARIABLES WITHIN EACH FIRM - use groupby
stock_prices['ret'] = stock_prices.groupby('Firm')['Adj Close'].pct_change()
stock_prices['ret'] = stock_prices['ret'] *100 # convert to p.p. to match FF's convention on scaling (daily % rets)
stock_prices

,Firm,Date,Adj Close,ret
0,AAPL,2016-01-04,24.251431,NaN
1,AAPL,2016-01-05,23.643709,-2.505920
2,AAPL,2016-01-06,23.181011,-1.956960
3,AAPL,2016-01-07,22.202665,-4.220462
4,AAPL,2016-01-08,22.320063,0.528753
...,...,...,...,...
751,SBUX,2016-12-23,51.753979,-0.175119
752,SBUX,2016-12-27,51.617821,-0.263087
753,SBUX,2016-12-28,51.154823,-0.896972
754,SBUX,2016-12-29,51.127586,-0.053244


In [7]:
# this shows that my "groupby" approach works, because 
# count of ret = count of adj close MINUS # of firms 
# meaning that the first day for each firm has no return data
stock_prices.describe() 

,Adj Close,ret
count,756.000000,753.000000
mean,42.032877,0.041001
std,12.780560,1.386916
min,21.036304,-7.171029
25%,25.972628,-0.604639
50%,48.533619,0.000000
75%,51.923299,0.759516
max,59.152493,6.496342


## Get the factor returns 

Above, we got the asset returns, $r_i$. 

To estimate $\alpha$ and $\beta$ in $r_i-r_f = \alpha + \beta (r_m-r_f)$, we need $(r_m-r_f)$ and $r_f$. Let's download those now.

Note: $(r_m-r_f)$ is the excess return on the market, which is _one_ "factor". Modern asset pricing typically uses 5 factors in tests.


In [8]:
# We need (r_mkt - rf), and rf
# the Fama French data library is a benchmark asset pricing dataset 
ff = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3_daily',start=start,end=end)[0] # the [0] is because the imported obect is a dictionary, and key=0 is the dataframe
ff = ff.reset_index().rename(columns={"Mkt-RF":"mkt_excess"})
ff

,Date,mkt_excess,SMB,HML,RMW,CMA,RF
0,2016-01-04,-1.59,-0.76,0.52,0.35,0.40,0.000
1,2016-01-05,0.12,-0.24,0.01,0.07,0.31,0.000
2,2016-01-06,-1.35,-0.22,0.00,0.16,0.05,0.000
3,2016-01-07,-2.44,-0.28,0.09,0.52,0.36,0.000
4,2016-01-08,-1.11,-0.51,-0.04,0.25,0.06,0.000
...,...,...,...,...,...,...,...
247,2016-12-23,0.19,0.52,-0.50,-0.40,-0.15,0.001
248,2016-12-27,0.27,0.23,0.14,0.14,0.04,0.001
249,2016-12-28,-0.87,-0.29,0.09,0.19,-0.15,0.001
250,2016-12-29,-0.04,0.10,-0.33,0.27,0.02,0.001


## Merge the asset and factor returns 

In [9]:
assets_and_factors = pd.merge(
    left=stock_prices,
    right=ff,
    on="Date",
    how="inner",
    indicator=True,
    validate="many_to_one",
)
assets_and_factors

,Firm,Date,Adj Close,ret,mkt_excess,SMB,HML,RMW,CMA,RF,_merge
0,AAPL,2016-01-04,24.251431,NaN,-1.59,-0.76,0.52,0.35,0.40,0.000,both
1,MSFT,2016-01-04,49.591057,NaN,-1.59,-0.76,0.52,0.35,0.40,0.000,both
2,SBUX,2016-01-04,52.108955,NaN,-1.59,-0.76,0.52,0.35,0.40,0.000,both
3,AAPL,2016-01-05,23.643709,-2.505920,0.12,-0.24,0.01,0.07,0.31,0.000,both
4,MSFT,2016-01-05,49.817295,0.456208,0.12,-0.24,0.01,0.07,0.31,0.000,both
...,...,...,...,...,...,...,...,...,...,...,...
751,MSFT,2016-12-29,58.483055,-0.142859,-0.04,0.10,-0.33,0.27,0.02,0.001,both
752,SBUX,2016-12-29,51.127586,-0.053244,-0.04,0.10,-0.33,0.27,0.02,0.001,both
753,AAPL,2016-12-30,27.254812,-0.779585,-0.52,-0.06,0.20,-0.09,0.03,0.001,both
754,MSFT,2016-12-30,57.776413,-1.208285,-0.52,-0.06,0.20,-0.09,0.03,0.001,both


## Estimate CAPM

So the data’s basically ready. _(We need to do two quick things below.)_

Again, the goal is to estimate, **for each** firm, **for each** year, alpha and beta, from the CAPM formula.

Well, as we've said, if you are doing a "for each" on a dataframe, that means you want to use groupby!

So, I have a dataframe, and **for each** firm, and **for each** year, I want to \<do stuff> (run regressions).

That almost directly translates to the code we need: `assets_and_factors.groupby([firm,year]).runregression()`. Except there is no "runregression" function that applies to pandas groupby objects. But we can write such a function and then `apply()` it. Meaning, our plan is to basically use this code: `assets_and_factors.groupby([firm,year]).apply(<our own reg fcn>)`.

We just need to write a reg function that works on groupby objects.

In [10]:
import statsmodels.api as sm


def reg_in_groupby(df, formula="ret_excess ~ mkt_excess + SMB + HML"):
    """
    Want to run regressions after groupby? E.g., repeat the regression 
    for each firm-year?
    
    This will do it! 
    
    Note: This defaults to a FF3 model assuming specific variable names. If you
    want to run any other regression, just specify your model.
    
    Usage: 
        df.groupby(<whatever>).apply(reg_in_groupby)
        df.groupby(<whatever>).apply(reg_in_groupby,formula=<whatever>)
    """
    return pd.Series(sm.formula.ols(formula, data=df).fit().params)

In [11]:
(
    assets_and_factors # grab the data
    
    # Two things before the regressions:
    # 1. need a year variable (to group on)
    # 2. the market returns in FF are excess returns, so 
    #    our stock returns need to be excess as well
    .assign(year = assets_and_factors.Date.dt.year,
            ret_excess = assets_and_factors.ret - assets_and_factors.RF)
    
    # ok, run the regs, so easy!
    .groupby(['Firm','year']).apply(reg_in_groupby,formula='ret_excess ~ mkt_excess')
    
    # and clean up - with better var names
    .rename(columns={'Intercept':'alpha','mkt_excess':'beta'})
    .reset_index()
)

,Firm,year,alpha,beta
0,AAPL,2016,-0.001081,0.965623
1,MSFT,2016,0.000690,1.166076
2,SBUX,2016,-0.056862,0.848163
